<a href="https://colab.research.google.com/github/JungCesar/bscaithesis/blob/master/bsc_ai_thesis_mfcc_MLPClassifier_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -V

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [2]:
%%capture
# !pip3 install librosa==0.6.3 numpy soundfile==0.9.0 sklearn PyAudio==0.2.11
!pip3 install librosa numpy soundfile scikit-learn

In [3]:
# Get access to personal Google Drive account
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# str(path) returns something like: /content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0002.wav
# tqdm is used to create a smart progress bar for the loops, for example it shows loading time

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

from pathlib import Path
from tqdm import tqdm

data = []

for path in tqdm(Path("/content/drive/MyDrive/bsc-ai-thesis/torgo_data").glob("**/*.wav")):
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('/')[-2]

    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "filename": name,
            "path": str(path),
            "dysarthria_class": label
        })

    except Exception as e:
        print(str(path), e)
        pass

187it [00:00, 446.60it/s]

/content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0068.wav Failed to open the input "/content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0068.wav" (Invalid data found when processing input).


2000it [00:05, 367.49it/s]


In [5]:
# Show how the Pandas dataframe looks like currently
import pandas as pd
df = pd.DataFrame(data)
df.head()

,filename,path,dysarthria_class
0,F01_Session1_0006,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female
1,F01_Session1_0038,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female
2,F01_Session1_0015,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female
3,F01_Session1_0024,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female
4,F01_Session1_0053,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria_female


In [6]:
import os

# Filter broken and non-existed paths
print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["status"])
df = df.drop(columns="status")

print(f"Step 1: {len(df)}")

df = df.sample(frac=1)
df = df.reset_index(drop=True)

Step 0: 1999
Step 1: 1999


In [7]:
# Eliminate difference between male and female and print distribbution
df = df.replace({'dysarthria_class' : {'dysarthria_female': 'dysarthria', 'dysarthria_male': 'dysarthria', 'non_dysarthria_female': 'non_dysarthria', 'non_dysarthria_male': 'non_dysarthria'}})
print("Labels: ", df["dysarthria_class"].unique())
print()
df.groupby('dysarthria_class').count()

Labels:  ['non_dysarthria' 'dysarthria']



,filename,path
dysarthria_class,,
dysarthria,999,999
non_dysarthria,1000,1000


In [8]:
df.head()

,filename,path,dysarthria_class
0,MC04_Session1_0257,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,non_dysarthria
1,F04_Session2_0145,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria
2,MC01_Session1_0065,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,non_dysarthria
3,F01_Session1_0132,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria
4,M05_Session2_0352,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,dysarthria


In [9]:
print(df['path'][0])

/content/drive/MyDrive/bsc-ai-thesis/torgo_data/non_dysarthria_male/MC04_Session1_0257.wav


In [10]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [11]:
import librosa

# Count how many samples there are of less than 1 second long
# In the next section these will be removed
lstsmall = []
lstlarge = []

for i in range(len(df['path'])):
    if librosa.get_duration(path=df['path'][i]) < 1:
        lstsmall.append(i)
    else:
        lstlarge.append(i)

print(len(lstsmall), lstsmall)
print(len(lstlarge), lstlarge)

32 [4, 79, 87, 106, 159, 204, 217, 252, 263, 494, 537, 541, 696, 727, 775, 930, 939, 1067, 1173, 1247, 1371, 1380, 1486, 1531, 1618, 1662, 1685, 1728, 1883, 1888, 1936, 1937]
1967 [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 

In [12]:
# import librosa

# # Filter broken and non-existed paths
# print(f"Step 0: {len(df)}")

# # Audio files with length of less than 1 second cause error for MFCC extraction
# # Therefore delete these files
# df["status"] = df["path"].apply(lambda x: True if (librosa.get_duration(path=x) > 1) else None)
# df = df.dropna(subset=["status"])
# df = df.drop(columns="status")

# print(f"Step 1: {len(df)}")

# # df = df.sample(frac=1)
# df = df.reset_index(drop=True)

In [13]:
import soundfile # to read audio file
import numpy as np
from sklearn.model_selection import train_test_split

# everyone takes mean of transpose?
def extract_mfcc(filename):
    y, sr  = librosa.load(filename)

    # mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=30).T, axis=0)
    # return mfcc

    # What is the difference of taking the mean of the transpose here instead of just keeping MFCC?????
    # Each coefficient will be averaged over time.
    # Make sure successive windows overlap slightly
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, n_fft=512, center=True).T, axis=0)
    return mfcc

    # return librosa.feature.mfcc(y=y, sr=sr, n_fft=512, center=True)

In [14]:
test_size=0.2
X, y = [], []
df = df.reset_index()  # make sure indexes pair with number of rows
for _, row in df.iterrows():
    # print(row['path'], row['dysarthria_class'])
    # get the base name of the audio file
    # basename = os.path.basename(file)
    # extract speech features
    features = extract_mfcc(row['path'])
    # get the emotion label
    label = row['dysarthria_class']
    # add to data
    X.append(features)
    y.append(label)

# Comment on UserWarning below:
# either pad the frame or reduce the FFT size. Note that frames are padded by default,
# in which case this is a warning (not an exception).
# tts = train_test_split(np.array(X), y, test_size=test_size)

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=512 is too large for input signal of length=185
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=512 is too large for input signal of length=209
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=512 is too large for input signal of length=214
  warnings.warn(


In [15]:
# for i in range(len(X)):
#     print()

i = 0
for arr in X:
    print(type(arr))
    print(arr.shape)
    if i == 5:
        break
    i += 1

<class 'numpy.ndarray'>
(20,)
<class 'numpy.ndarray'>
(20,)
<class 'numpy.ndarray'>
(20,)
<class 'numpy.ndarray'>
(20,)
<class 'numpy.ndarray'>
(20,)
<class 'numpy.ndarray'>
(20,)


In [16]:
print(type(X))

<class 'list'>


In [17]:
print(len(y))
print(y)

1999
['non_dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria', 'dysarthria', 'dysarthria', 'dysarthria', 'dysarthria', 'dysarthria', 'non_dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'non_dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'non_dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria', 'dysarthria', 'dysarthria', 'non_dysarthria', 'non_dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria', 'dysarthria', 'non_dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'non_dysarthria', 'non_dysarthria', 'non_dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'non_dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria', 'dysarthria', 'dysarthria', 'non_dysarthria', 'dysarthria'

In [18]:
# load RAVDESS dataset, 75% training 25% testing
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size=0.2)

In [19]:
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted
# using extract_features() function
print("[+] Number of features:", X_train.shape[1])

[+] Number of training samples: 1599
[+] Number of testing samples: 400
[+] Number of features: 20


In [20]:
# best model, determined by a grid search
model_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08,
    'hidden_layer_sizes': (300,),
    'learning_rate': 'adaptive',
    'max_iter': 500,
}

In [21]:
from sklearn.neural_network import MLPClassifier # multi-layer perceptron model

# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
# Do this in Keras or PyTorch
model = MLPClassifier(**model_params)

In [22]:
# train the model
print("[*] Training the model...")
clf = model.fit(X_train, y_train)
print(clf)

[*] Training the model...
MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
precision = precision_score(y_true=y_test, y_pred=y_pred, labels=['dysarthria', 'non_dysarthria'], pos_label='dysarthria')
recall = recall_score(y_true=y_test, y_pred=y_pred, labels=['dysarthria', 'non_dysarthria'], pos_label='dysarthria')
f1 = f1_score(y_true=y_test, y_pred=y_pred, labels=['dysarthria', 'non_dysarthria'], average='binary', pos_label='dysarthria')
roc_auc = roc_auc_score(y_true=y_test,  y_score=y_proba[:, 1])

# Prints measurements
print("Accuracy: {:.2f}%".format(accuracy*100))
print("Precision: {:.2f}%".format(precision*100))
print("Recall: {:.2f}%".format(recall*100))
print("F1-Score: {:.2f}%".format(f1*100))
print("AUC_ROC: {:.2f}%".format(roc_auc*100))

Accuracy: 83.75%
Precision: 78.24%
Recall: 93.50%
F1-Score: 85.19%
AUC_ROC: 93.79%


In [24]:
# def load_data(test_size=0.2):
#     X, y = [], []
#     df = data.reset_index()  # make sure indexes pair with number of rows
#     for _, row in df.iterrows():
#         # print(row['path'], row['dysarthria_class'])
#         # get the base name of the audio file
#         # basename = os.path.basename(file)
#         # extract speech features
#         features = extract_mfcc(row['path'])
#         # get the emotion label
#         label = row['dysarthria_class']
#         # add to data
#         X.append(features)
#         y.append(label)
#     # split the data to training and testing and return it
#     return train_test_split(np.array(X), y, test_size=test_size)